# Computational Theory Assessment

## Problem 1

In [3]:
import numpy as np

def parity(x, y, z):
    """
    Parity function: x ⊕ y ⊕ z
    
    Args:
        x, y, z: 32-bit integers
        
    Returns:
        32-bit integer result of XOR operation
    """
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    return np.uint32(x ^ y ^ z)

## Problem 2

## Problem 3

## Problem 4

## Problem 5